# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [50]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
# Total number of clicks and views
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [84]:
# Did anyone click who did not view? (anomaly check)
click_ids = set(df[df['action'] == 'click']['id'].unique())
view_ids = set(df[df['action'] == 'view']['id'].unique())

print("People who clicked but did not view (if any): {}".format(click_ids - view_ids))

People who clicked but did not view (if any): set()


In [4]:
# Size of control and experimental groups
df['group'].value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [6]:
# Experiment time frame
print("Minimum Timestamp:",df['timestamp'].min())
print("Maximum Timestamp:", df['timestamp'].max())

Minimum Timestamp: 2016-09-24 17:42:27.839496
Maximum Timestamp: 2017-01-18 10:24:08.629327


In [35]:
# Total clicks and views by group
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [85]:
#Is there any overlap between the control and experiment groups? 
    # If so, how plan to account for this in experrimental design
experiment_ids = set(df[df['group'] == 'experiment']['id'].unique())
control_ids = set(df[df['group'] == 'control']['id'].unique())

print("People in experiement and control groups (if any): {}".format(experiment_ids & control_ids))

People in experiement and control groups (if any): set()


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

For this sencario, we would like to compare the frequency of 'clicks' for both the control and experimental groups.  A chi-squared goodness of fit test will be most appropriate.  

Null Hypothesis: People who viewed the experimental homepage were not more nor less likely to click on the page than those who viewed the control homepage.

Alternative Hypothesis: People who viewed the experimental homepage were more likely to click on the page than those who viewed the control homepage.

For this experiement, we will use an alpha level of 0.01.

In [38]:
df.groupby('group')['action'].value_counts()

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [43]:
counts = pd.DataFrame(df.groupby('group')['action'].value_counts())

control_clicks = int(counts.loc['control','click'])
control_views_only = int(counts.loc['control','view']) - control_clicks

experiment_clicks = int(counts.loc['experiment','click'])
experiment_views_only = int(counts.loc['experiment', 'view']) - experiment_clicks

In [48]:
contingency_table = np.array([(control_clicks, control_views_only),
                             (experiment_clicks, experiment_views_only)])

In [54]:
chi, pval, dof, expected = stats.chi2_contingency(contingency_table)
print("The chi-squared statistic is ", round(chi, 4))
print("The p-value is ", round(pval, 4))

The chi-squared statistic is  6.7129
The p-value is  0.0096


From the above results we can see that our p-value is lower than our assigned alpha of 0.01.  This means we can reject the null hypothesis at this significance level.

In [55]:
# Difference in click likelyhood of a click by percentage
contingency_table

array([[ 932, 2400],
       [ 928, 2068]])

In [65]:
percentages = (contingency_table[:,0]/contingency_table[:,1])*100
abs_diff = abs(percentages[0] - percentages[1])

In [64]:
print("Click rate for the control homepage: {}%".format(round(percentages[0],2)))
print("Click rate for the experimental homepage: {}%".format(round(percentages[1],2)))
print("Absolute difference of rates: {}%".format(round(abs_diff, 2)))

Click rate for the control homepage: 38.83%
Click rate for the experimental homepage: 44.87%
Absolute difference of rates: 6.04%


The effect size observed is a increase of 6.04% of clicks for the experimental homepage.

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [68]:
experiment_views_total = int(counts.loc['experiment', 'view'])
control_views_total = int(counts.loc['control', 'view'])
expected_clicks = (control_clicks/control_views_total) * experiment_views_total
print("Expected number of clicks: ", round(expected_clicks,2))

Expected number of clicks:  838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [72]:
ratio = control_clicks/control_views_total
control_variance = experiment_views_total * ratio * (1 - ratio)
std = np.sqrt(control_variance)

z_score = (experiment_clicks - expected_clicks)/std
z_score

3.6625360854823588

The actual number of clicks is 3.66 standard deviations away from the expected number of clicks based on the rate of clicks from the control homepage.

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [75]:
pval = 1 - stats.norm.cdf(z_score)
pval

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

The result does indeed roughly match that of the previous statistical test.  While this p-value is smaller than the statistic found earlier, both lead to the same conclusion of rejecting the null hypothesis. We can conclude that the new homepage does indeed lead to an increase in clicks.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.